In [6]:
import csv
from datetime import datetime
from functions import *
import json
import multiprocessing
import pandas as pd
import psycopg2
from psycopg2 import extras as ext
import sys
import time
import tweepy
from tweepy.auth import OAuthHandler

configuration = json.load(open("configuration.json"))

# Pull a user's data from the Twitter API

In [5]:
def pull_user_data(api, user_ids):
    if len(user_ids) > 100:
        raise ValueError("Maximum 100 user_ids per request.")
    
    try:
        collected_at = datetime.now()
        users_data = api.lookup_users(user_ids=user_ids)
        users_data = [(user_data._json, collected_at) for user_data in users_data]
        
    except tweepy.RateLimitError:
        time.sleep(15 * 60)
        users_data = pull_user_data(api, user_ids)
    
    return users_data


# Write the user data
Currently written for Postgres

In [7]:
def write_users_to_database(configuration, users_data, start_interval, end_interval):
    config_file = configuration["db_config_file"]
    for line in open(db_config_file).readlines():
        key, value = line.strip().split("=")
        config[key] = value
    
    table_name = config_file["table_prefix"] + "_user_data"
    
    ## Connect to the database and get a cursor object
    database = psycopg2.connect(**config)
    cursor = database.cursor()

    ## Create the table if it doesn't exist
    CREATE_STATEMENT = """CREATE TABLE IF NOT EXISTS {table_name} (
                          user_id BIGINT,
                          collected_at TIMESTAMP,
                          user_data JSON,
                          start_interval TIMESTAMP,
                          end_interval TIMESTAMP);"""
    
    cursor.execute(CREATE_STATEMENT)
    
    ## Insert the rows
    INSERT_STATEMENT = """INSERT INTO {table_name} (user_id, collected_at, user_data, start_interval, end_interval) 
                          VALUES (%s, %s, %s, %s, %s)""".format(table_name=table_name)
    
    rows = [(u[0]["id"], u[1], u[0], start_interval, end_interval) for u in users_data]
    
    ext.execute_batch(cursor, INSERT_STATEMENT, rows)
    
    database.commit()